In [1]:
cd ..

/Users/satouwataru/Desktop/DiveIntoCode/git/diveintocode-ml/GraduationWork/v2/codes/model


In [2]:
cd ..

/Users/satouwataru/Desktop/DiveIntoCode/git/diveintocode-ml/GraduationWork/v2/codes


In [3]:
cd ..

/Users/satouwataru/Desktop/DiveIntoCode/git/diveintocode-ml/GraduationWork/v2


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# import preprocessing
import rightgbm_category
import rightgbm_regression

def predict_toto(year = None):
    
    acuracy_val_list = []
    acuracy_title_list = []
    
    model_category =  rightgbm_category.rightgbm_category()
    model_regression =  rightgbm_regression.rightgbm_regression()
    
    data = pd.read_csv('data/model/preprocessing.csv', index_col=0).reset_index(drop = True)
    data = data.sort_values('年月日', ascending=False)
    # y_columns = ['y_H_goal', 'y_A_goal', 'y_goal_deff', 'y_even_flg', 'y_H_result']
    y_columns = []
    for col in data.columns:
        if col[:2] == 'y_':
            y_columns += [col]
    
    '''
    引き分け、その他の二値分類
    '''
    # train、test、totoに分ける
    train = data[data['train_test'] == 'train'].drop(columns = ['train_test'])
    test = data[data['train_test'] == 'test'].drop(columns = ['train_test'])
    toto = data[data['train_test'] == 'toto'].drop(columns = ['train_test'])
    if test.shape[0] == 0:
        train, test = train_test_split(train, test_size=0.05)
    
    # trainとvalに分ける
    train, val = train_test_split(train, train_size=0.8)

    train = train.sort_values('年月日', ascending=False).reset_index(drop = True)
    val = val.sort_values('年月日', ascending=False).reset_index(drop = True)
    test = test.sort_values('年月日', ascending=False).reset_index(drop = True)
    toto = toto.sort_values('年月日', ascending=False).reset_index(drop = True)
    
    # x_とy_に分ける
    x_train = train.drop(columns = y_columns)
    x_val = val.drop(columns = y_columns)
    x_test = test.drop(columns = y_columns)
    x_toto = toto.drop(columns = y_columns)
    y_train = train['y_even_flg']
    y_val = val['y_even_flg']
    y_test = test['y_even_flg']
    
    df_result = x_toto[['年月日', 'H_Team', 'A_Team']]
    
    # 学習
    y_pred, y_pred_proba, accuracy = model_category.fit(x_train, x_val, x_test, y_train, y_val, y_test, 2, year, 'is_even_lgbm')
    # 予測
    y_toto, y_toto_proba = model_category.predict(x_toto)
    
    df_pred_even = pd.DataFrame(y_toto, columns = ['is_even_lgbm'])
    df_pred_even_prova = pd.DataFrame(y_toto_proba, columns = ['is_even_0_1_lgbm', 'is_even_1_1_lgbm'])
    df_pred_even_prova['is_even_lgbm_acc'] = accuracy
    df_result = pd.concat([df_result, df_pred_even, df_pred_even_prova], axis = 1)
    acuracy_val_list += [accuracy]
    acuracy_title_list += ['is_even_lgbm']
    
    '''
    勝敗の二値分類
    '''
    # y_columns = ['y_H_goal', 'y_A_goal', 'y_goal_deff', 'y_even_flg', 'y_H_result']
    # train、test、totoに分ける
    train = data[data['train_test'] == 'train'].drop(columns = ['train_test'])
    test = data[data['train_test'] == 'testa'].drop(columns = ['train_test'])
    if test.shape[0] == 0:
        train, test = train_test_split(train, test_size=0.05)
    
    # trainの勝敗のみを使用
    train = train[train['y_H_result'] != 0]
    
    # trainとvalに分ける
    train, val = train_test_split(train, train_size=0.8)

    train = train.sort_values('年月日', ascending=False).reset_index(drop = True)
    val = val.sort_values('年月日', ascending=False).reset_index(drop = True)
    test = test.sort_values('年月日', ascending=False).reset_index(drop = True)
    
    train['y_H_result'] = train['y_H_result'] -1
    val['y_H_result'] = val['y_H_result'] -1
    test['y_H_result'] = test['y_H_result'] -1
    
    # x_とy_に分ける
    x_train = train.drop(columns = y_columns)
    x_val = val.drop(columns = y_columns)
    x_test = test.drop(columns = y_columns)
    y_train = train['y_H_result']
    y_val = val['y_H_result']
    y_test = test['y_H_result']
    
    # 学習
    y_pred, y_pred_proba, accuracy = model_category.fit(x_train, x_val, x_test, y_train, y_val, y_test, 2, year, 'is_lose_lgbm')
    # 予測
    y_toto, y_toto_proba = model_category.predict(x_toto)
    # 結果を保存
    df_pred = pd.DataFrame(y_toto, columns = ['is_lose_lgbm'])
    df_pred_prova = pd.DataFrame(y_toto_proba, columns = ['is_lose_0_1_lgbm', 'is_lose_1_1_lgbm'])
    df_pred_prova['is_lose_lgbm_acc'] = accuracy
    df_result = pd.concat([df_result, df_pred, df_pred_prova], axis = 1)
    acuracy_val_list += [accuracy]
    acuracy_title_list += ['is_lose_lgbm']
    
    '''
    勝分敗の3値分類
    '''
    # y_columns = ['y_H_goal', 'y_A_goal', 'y_goal_deff', 'y_even_flg', 'y_H_result']
    # train、test、totoに分ける
    train = data[data['train_test'] == 'train'].drop(columns = ['train_test'])
    test = data[data['train_test'] == 'testa'].drop(columns = ['train_test'])
    if test.shape[0] == 0:
        train, test = train_test_split(train, test_size=0.05)
        
    # trainとvalに分ける
    train, val = train_test_split(train, train_size=0.8)

    train = train.sort_values('年月日', ascending=False).reset_index(drop = True)
    val = val.sort_values('年月日', ascending=False).reset_index(drop = True)
    test = test.sort_values('年月日', ascending=False).reset_index(drop = True)
    
    # x_とy_に分ける
    x_train = train.drop(columns = y_columns)
    x_val = val.drop(columns = y_columns)
    x_test = test.drop(columns = y_columns)
    y_train = train['y_H_result']
    y_val = val['y_H_result']
    y_test = test['y_H_result']
    
    # 学習
    y_pred, y_pred_proba, accuracy = model_category.fit(x_train, x_val, x_test, y_train, y_val, y_test, 3, year, 'w_e_l_lgbm')
    # 予測
    y_toto, y_toto_proba = model_category.predict(x_toto)
    # 結果を保存
    df_pred = pd.DataFrame(y_toto, columns = ['w_e_l_lgbm'])
    df_pred_prova = pd.DataFrame(y_toto_proba, columns = ['w_e_l_0_2_lgbm', 'w_e_l_1_2_lgbm', 'w_e_l_2_2_lgbm'])
    df_pred_prova['w_e_l_lgbm_acc'] = accuracy
    df_result = pd.concat([df_result, df_pred, df_pred_prova], axis = 1)
    acuracy_val_list += [accuracy]
    acuracy_title_list += ['w_e_l_lgbm']
    
    '''
    得失点の回帰
    '''
    # y_columns = ['y_H_goal', 'y_A_goal', 'y_goal_deff', 'y_even_flg', 'y_H_result']
    # train、test、totoに分ける
    train = data[data['train_test'] == 'train'].drop(columns = ['train_test'])
    test = data[data['train_test'] == 'testa'].drop(columns = ['train_test'])
    if test.shape[0] == 0:
        train, test = train_test_split(train, test_size=0.05)
        
    # trainとvalに分ける
    train, val = train_test_split(train, train_size=0.8)

    train = train.sort_values('年月日', ascending=False).reset_index(drop = True)
    val = val.sort_values('年月日', ascending=False).reset_index(drop = True)
    test = test.sort_values('年月日', ascending=False).reset_index(drop = True)
    
    # x_とy_に分ける
    x_train = train.drop(columns = y_columns)
    x_val = val.drop(columns = y_columns)
    x_test = test.drop(columns = y_columns)
    y_train = train['y_goal_deff']
    y_val = val['y_goal_deff']
    y_test = test['y_goal_deff']
    
    # 学習
    y_pred, rmse = model_regression.fit(x_train, x_val, x_test, y_train, y_val, y_test, year, 'goal_deff_lgbm')
    # 予測
    y_toto = model_regression.predict(x_toto)
    # 結果を保存
    df_pred = pd.DataFrame(y_toto, columns = ['goal_deff_lgbm'])
    df_pred['goal_deff_lgbm_rmse'] = rmse
    df_result = pd.concat([df_result, df_pred], axis = 1)
    acuracy_val_list += [rmse]
    acuracy_title_list += ['goal_deff_lgbm']
    
    '''
    得点の回帰
    '''
    # y_columns = ['y_H_goal', 'y_A_goal', 'y_goal_deff', 'y_even_flg', 'y_H_result']
    # train、test、totoに分ける
    train = data[data['train_test'] == 'train'].drop(columns = ['train_test'])
    test = data[data['train_test'] == 'testa'].drop(columns = ['train_test'])
    if test.shape[0] == 0:
        train, test = train_test_split(train, test_size=0.05)
        
    # trainとvalに分ける
    train, val = train_test_split(train, train_size=0.8)

    train = train.sort_values('年月日', ascending=False).reset_index(drop = True)
    val = val.sort_values('年月日', ascending=False).reset_index(drop = True)
    test = test.sort_values('年月日', ascending=False).reset_index(drop = True)
    
    # x_とy_に分ける
    x_train = train.drop(columns = y_columns)
    x_val = val.drop(columns = y_columns)
    x_test = test.drop(columns = y_columns)
    y_train = train['y_H_goal']
    y_val = val['y_H_goal']
    y_test = test['y_H_goal']
    
    # 学習
    y_pred, rmse = model_regression.fit(x_train, x_val, x_test, y_train, y_val, y_test, year, 'H_goal_lgbm')
    # 予測
    y_toto = model_regression.predict(x_toto)
    # 結果を保存
    df_pred = pd.DataFrame(y_toto, columns = ['H_goal_lgbm'])
    df_pred['H_goal_lgbm_rmse'] = rmse
    df_result = pd.concat([df_result, df_pred], axis = 1)
    acuracy_val_list += [rmse]
    acuracy_title_list += ['H_goal_lgbm']
    
    '''
    失点の回帰
    '''
    # y_columns = ['y_H_goal', 'y_A_goal', 'y_goal_deff', 'y_even_flg', 'y_H_result']
    # train、test、totoに分ける
    train = data[data['train_test'] == 'train'].drop(columns = ['train_test'])
    test = data[data['train_test'] == 'testa'].drop(columns = ['train_test'])
    if test.shape[0] == 0:
        train, test = train_test_split(train, test_size=0.05)
        
    # trainとvalに分ける
    train, val = train_test_split(train, train_size=0.8)

    train = train.sort_values('年月日', ascending=False).reset_index(drop = True)
    val = val.sort_values('年月日', ascending=False).reset_index(drop = True)
    test = test.sort_values('年月日', ascending=False).reset_index(drop = True)
    
    # x_とy_に分ける
    x_train = train.drop(columns = y_columns)
    x_val = val.drop(columns = y_columns)
    x_test = test.drop(columns = y_columns)
    y_train = train['y_A_goal']
    y_val = val['y_A_goal']
    y_test = test['y_A_goal']
    
    # 学習
    y_pred, rmse = model_regression.fit(x_train, x_val, x_test, y_train, y_val, y_test, year, 'A_goal_lgbm')
    # 予測
    y_toto = model_regression.predict(x_toto)
    # 結果を保存
    df_pred = pd.DataFrame(y_toto, columns = ['A_goal_lgbm'])
    df_pred['A_goal_lgbm_rmse'] = rmse
    df_result = pd.concat([df_result, df_pred], axis = 1)
    acuracy_val_list += [rmse]
    acuracy_title_list += ['A_goal_lgbm']
    
#     df_result.drop(columns = ['even_1_lgbm', 'wl_1_lgbm', 'wel_2_lgbm'])
    df_acuracy = pd.DataFrame([acuracy_val_list], columns = acuracy_title_list).T
    return df_result, df_acuracy

In [5]:
df_result, df_acuracy = predict_toto()

[I 2021-05-11 09:41:34,527] A new study created in memory with name: no-name-bffe215b-83f3-40cd-8de7-93c224979e1d
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds


feature_fraction, val_score: 0.693140:  14%|#4        | 1/7 [00:00<00:05,  1.01it/s]

Early stopping, best iteration is:
[1]	valid_0's multi_logloss: 0.69314
Training until validation scores don't improve for 100 rounds


feature_fraction, val_score: 0.691434:  29%|##8       | 2/7 [00:01<00:04,  1.00it/s]

...

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[14]	valid_0's rmse: 1.09978
Training until validation scores don't improve for 100 rounds


min_data_in_leaf, val_score: 1.096757:  60%|######    | 3/5 [00:00<00:00,  5.42it/s]

Early stopping, best iteration is:
[11]	valid_0's rmse: 1.10326
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[14]	valid_0's rmse: 1.10623
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[17]	valid_0's rmse: 1.09987


min_data_in_leaf, val_score: 1.096757: 100%|##########| 5/5 [00:00<00:00,  5.11it/s]

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[13]	valid_0's rmse: 1.10312
rmse :  1.1273393009305084


In [6]:
pd.concat([df_result,df_acuracy], axis = 1)

,年月日,H_Team,A_Team,is_even_lgbm,is_even_0_1_lgbm,is_even_1_1_lgbm,is_even_lgbm_acc,is_lose_lgbm,is_lose_0_1_lgbm,is_lose_1_1_lgbm,...,w_e_l_1_2_lgbm,w_e_l_2_2_lgbm,w_e_l_lgbm_acc,goal_deff_lgbm,goal_deff_lgbm_rmse,H_goal_lgbm,H_goal_lgbm_rmse,A_goal_lgbm,A_goal_lgbm_rmse,0
0,20201101.0,40.0,22.0,0.0,0.517855,0.482145,0.442708,0.0,0.722023,0.277977,...,0.429665,0.344232,0.473958,0.107102,1.510898,1.534894,1.137032,0.864063,1.127339,NaN
1,20201101.0,33.0,39.0,1.0,0.472177,0.527823,0.442708,0.0,0.503258,0.496742,...,0.377315,0.396526,0.473958,-0.067474,1.510898,1.311225,1.137032,0.829646,1.127339,NaN
2,20201101.0,25.0,41.0,1.0,0.472411,0.527589,0.442708,0.0,0.700948,0.299052,...,0.486657,0.296261,0.473958,0.390572,1.510898,1.557606,1.137032,0.751652,1.127339,NaN
3,20201101.0,36.0,26.0,1.0,0.469054,0.530946,0.442708,0.0,0.560327,0.439673,...,0.407925,0.353188,0.473958,0.047902,1.510898,1.225003,1.137032,1.029955,1.127339,NaN
4,20201101.0,32.0,27.0,0.0,0.505576,0.494424,0.442708,0.0,0.591624,0.408376,...,0.452884,0.322239,0.473958,0.234438,1.510898,1.522925,1.137032,1.601814,1.127339,NaN
5,20201101.0,35.0,21.0,1.0,0.457609,0.542391,0.442708,0.0,0.514140,0.485860,...,0.329249,0.435178,0.473958,-0.073798,1.510898,1.351944,1.137032,1.083416,1.127339,NaN
6,20201031.0,1.0,15.0,0.0,0.500339,0.499661,0.442708,0.0,0.523654,0.476346,...,0.398877,0.407158,0.473958,0.010810,1.510898,1.351132,1.137032,1.427625,1.127339,NaN
7,20201031.0,19.0,3.0,1.0,0.483529,0.516471,0.442708,1.0,0.405208,0.594792,...,0.334080,0.482137,0.473958,-0.092950,1.510898,1.340565,1.137032,1.833370,1.127339,NaN
8,20201031.0,9.0,8.0,1.0,0.486102,0.513898,0.442708,0.0,0.550414,0.449586,...,0.397965,0.423773,0.473958,0.092725,1.510898,1.424891,1.137032,1.616674,1.127339,NaN
9,20201031.0,2.0,11.0,1.0,0.476583,0.523417,0.442708,0.0,0.540133,0.459867,...,0.381114,0.463194,0.473958,-0.059919,1.510898,1.563272,1.137032,1.425202,1.127339,NaN


In [7]:
df_acuracy

,0
is_even_lgbm,0.442708
is_lose_lgbm,0.479167
w_e_l_lgbm,0.473958
goal_deff_lgbm,1.510898
H_goal_lgbm,1.137032
A_goal_lgbm,1.127339
